In [ ]:
import os
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from astropy.table import Table
from astropy.io import fits
from PIL import Image
import warnings
import glob
warnings.filterwarnings('ignore')

In [ ]:
homedir = os.getenv('HOME')

sys.path.append(homedir+'/github/halphagui/')
import photwrapper
from image_functions import *

cwd = homedir+'/Desktop/gbtresearch/'
os.chdir(cwd)

In [ ]:
datapath = homedir+'/Desktop/gbtresearch/data/gbt22a_430_sample_legacy.fits'
gbt = Table.read(datapath); gbt
gbtTest=gbt[0:4]; gbtTest

In [ ]:
def getWiseFits(myGal,plotting=False,imsize=120,makeMask=True,changeSize=False):
    '''
    Reads in a table of galaxy coordinates to get wise/legacy images to calculate photometry; W1 phot will be converted
    into a flux, then converted into a stellar mass.
    
    Inputs:
        myGal = table entry of galactic coordinate data; needs 'name,' 'RA,' and 'DEC' columns
        plotting = conditional to plot each photwrapper image step; default set to False
        imsize = length/width of legacy/WISE image cutout; default set to 120; units: pixels
        makeMask = conditional to mask off subtracted sky objects from segmentation image; default set to True
        changeSize = conditional to search for previously made unwise images, deletes them, and remakes them with given imsize; 
                        default set to False
    
    RETURNS:
        ptab = table created from output photometry file via photwrapper
        e = photwrapper ellipse data
    '''
    
    # get sky coords and galaxy AGC ID from input table
    ra = myGal['RA']
    dec = myGal['DEC']
    galname = f"AGC{myGal['name']:06d}"
    
    # setting pixel scaling
    UNWISE_PIXSCALE = 2.75
    LEGACY_PIXSCALE = 1
    
    if changeSize:
        # checks to see if unwise images were created
        dstring=f'{galname}-unwise*'
        flist=glob.glob(dstring)
        # removes image if it exists in the directory
        for f in flist:
            if os.path.exists(f):
                os.remove(f)
    
    # gets the W1-4 and legacy images for the galaxy
    legacyImgs, wiseImgs = display_legacy_unwise(ra,dec,galname,imsize_arcsec=imsize)
    
    # can be changed to different bands
    imname = wiseImgs[0]
    
    %matplotlib inline
    
    if plotting:
    
        print()
    
        plt.figure(figsize=(12,6.5))

        # concatinate lists
        imnames = ['grz','g','r','z']
        # plot legacy images in top row
        for i,im in enumerate(legacyImgs):
            plt.subplot(2,4,i+1)
            if i == 0:
                # display jpg
                t = Image.open(im)
                plt.imshow(t,origin='upper')
            else:
                data = fits.getdata(im)
                display_image(data,lowrange=False,percent=95)
            plt.title(imnames[i],fontsize=14)

        # plot WISE images
        imnames = ['W1','W2','W3','W4']
        for i,im in enumerate(wiseImgs):
            plt.subplot(2,4,4+i+1)
            data = fits.getdata(im)
            display_image(data,percent=92)
            plt.title(imnames[i],fontsize=14)
            
        hdu = fits.open(imname)
        data = hdu[0].data
        hdu.close()
    
        plt.figure()
        plt.title(f'{imname}')
        display_image(data)
    
    ### PHOTWRAPPER ####
        
    e = photwrapper.ellipse(imname)
    
    e.detect_objects()
        
    test = e.cat.to_table()
    
    e.find_central_object()
    
    if makeMask:
        e.get_mask_from_segmentation()

    e.get_ellipse_guess()
    
    print(); e.measure_phot()     
    e.calc_sb()
    e.convert_units()
    
    print(); e.write_phot_tables()
    e.write_phot_fits_tables(); print()
        
    if plotting:
        
        e.show_seg_aperture()
        e.draw_guess_ellipse_mpl()
        e.draw_phot_apertures()
    
    
    ptabName = imname.replace('.fits','-phot.fits')
    ptab = Table.read(ptabName)
     
    plt.figure(figsize=(12,5))
    
    plt.subplot(1,2,1)
    plt.plot(ptab['sma_arcsec'],ptab['flux'])
    plt.title(f'{galname} $F_{{enc}}$ Vs. SMA',fontsize=16)
    plt.xlabel("Semi-major axis (arcsec)",fontsize=14)
    plt.ylabel("$F_{enc}$ (erg / $cm^2$ s)",fontsize=14)
    
    plt.subplot(1,2,2)
    plt.plot(ptab['sma_arcsec'],ptab['mag'])
    plt.title(f'{galname} Magnitude Vs. SMA',fontsize=16)
    plt.xlabel("Semi-major axis (arcsec)",fontsize=14)
    plt.ylabel("Magnitude",fontsize=14)
    
    print(f"Max enclosed flux: {np.max(ptab['flux']):.2e}")
    print(f"Max SMA: {np.max(ptab['sma_arcsec'])}"); print()
    
    return ptab,e

In [ ]:
ptab,e = getWiseFits(
    
    gbt[4], 
    
    plotting=True,
    
    imsize=180, 
    
    makeMask=True,
    
    changeSize=True
    
)

In [ ]:
e.cat.to_table()

In [ ]:
ptab

# Testing function

In [ ]:
ptab,e = getWiseFits(gbt[16],plotting=True,imsize=180,changeSize=False)
print(np.max(ptab['flux']),np.max(ptab['sma_arcsec']))

In [ ]:
import scipy.ndimage as ndi

In [ ]:
segmap=e.segmentation.data == e.cat.label[e.objectIndex]
plt.figure()
plt.imshow(segmap)

mask_data = e.segmentation.data - segmap*e.cat.label[e.objectIndex]
plt.figure()
plt.imshow(mask_data)

segmap_float = ndi.uniform_filter(np.float64(mask_data),size=10)
plt.figure()
plt.imshow(segmap_float)

mask = segmap_float > 0.1
plt.figure()
plt.imshow(5*mask+segmap)

In [ ]:
e.cat.to_table()
e.cat.segment_flux[0]

In [ ]:
ptab

In [ ]:
e.plot_profiles()

In [ ]:
ptab = getLegacyfits(gbt[6],plotting=True)

In [ ]:
ptab

In [ ]:
e.segmentation

In [ ]:
UNWISE_PIXSCALE = 2.75
LEGACY_PIXSCALE = 1
        
# gets the W1-4 .fits files as well as the legacy images for the galaxy
display_legacy_unwise(ra,dec,galname,imsize_arcsec=120)

imsize_arcsec = 120 # length/width of image in arcseconds
imsize_pixels_legacy = round(imsize_arcsec/LEGACY_PIXSCALE)
imsize_pixels_unwise = round(imsize_arcsec/UNWISE_PIXSCALE)
    
# sorts the noise files from the image files that were previously downloaded
t = get_unwise_image(ra,dec,galid=galname,makeplots=False,imsize=str(imsize_pixels_unwise))
imagefiles = t[0]
noisefiles = t[1]
imagefiles.sort()
noisefiles.sort()

In [ ]:
e = photwrapper.ellipse(imname)


In [ ]:
%load_ext autoreload
%autoreload 2
            
imname = f'{galname}-legacy-120-r.fits'

    
e = photwrapper.ellipse(imname)
    
e.detect_objects()
    
test = e.cat.to_table()
    
e.find_central_object()
e.get_ellipse_guess()
e.measure_phot()     
e.calc_sb()
e.convert_units()
    
e.write_phot_tables()
e.write_phot_fits_tables(); print()
        
e.show_seg_aperture()
e.draw_guess_ellipse_mpl()
e.draw_phot_apertures()

ptab = Table.read(f'{galname}-legacy-120-r-phot.fits')

In [ ]:
e.segmentation

In [ ]:
plt.imshow(e.image,origin='lower')

In [ ]:
from photutils import detect_threshold, detect_sources

# changing to remove deprecated function source_properties
#from photutils import source_properties
from photutils.segmentation import SourceCatalog

In [ ]:
seg=detect_sources(e.image)

In [ ]:
e.threshold